### Basic setup

In [ ]:
!pip3 install -r requirements.txt

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.4 MB/s eta 0:00:00
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 3.3 MB/s eta 0:00:0000:0100:01
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.6/463.6 kB 4.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 3.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 3.7 MB/s eta 0:00:0000:0100:01
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 3.3

In [112]:
# Imports
import sys
import fitz  
import ollama 
import psycopg2 
import numpy as np
from pgvector.psycopg2 import register_vector
from datetime import datetime

In [117]:
"""
1. Base llm call
ollama pull mistral:7b or ollama pull qwen2.5:7b
"""
MODEL = "llama3.2:3b"  
# mistral:7b, qwen2.5:7b , qwen3:7b

def chat(msg):
    resp = ollama.chat(model=MODEL, messages=[{"role": "user", "content": msg}])
    answer = resp["message"]["content"]
    return answer

print(f"You are now chating with {MODEL} (say 'exit' to quit)\n")
hist = []
while True:
    msg = input("You: ").strip()
    if msg.lower() in ["quit", "exit", "Exit"]: break
    if msg: 
        print("You: ", msg)
        print(f"{MODEL}: {chat(msg)}\n")

You are now chating with llama3.2:3b (say 'exit' to quit)

You:  helo, what is your name
llama3.2:3b: Hello! I don't have a personal name, but I'm an AI designed to assist and communicate with users. You can call me "Assistant" or "AI" if you prefer. How can I help you today?

You:  okay, so your name now is bob, ok?
llama3.2:3b: Hi there! Nice to meet you. So, I'm Bob from now on. How can I assist you today?

You:  what is your name?
llama3.2:3b: I don't have a personal name, but I'm an AI designed to assist and communicate with users. You can think of me as a conversational companion or a helpful tool, rather than a person with a specific name. Some people refer to me as "Assistant" or "AI" for short. What's your name?



In [132]:
"""

"""
MODEL = "llama3.2:3b"  # mistral:7b, qwen2.5:7b , qwen3:7b

def chat(msg, history=[]):
    history.append({"role": "user", "content": msg})
    resp = ollama.chat(model=MODEL, messages=history)
    anwser = resp["message"]["content"]
    history.append({"role": "assistant", "content": anwser})
    return anwser

print(f" Chating with {MODEL} \n")
hist = []
while True:
    msg = input("You: ").strip()
    if msg.lower() in ["sair", "exit"]: break
    if msg: 
        print("You: ", msg)
        print(f"Bot: {chat(msg, hist)}\n")

 Chating with llama3.2:3b 

You:  helo bob
Bot: Hello! It's nice to meet you, but I'm not actually Bob. I'm an AI assistant designed to help and chat with users like you. What can I do for you today?



In [119]:
"""
Chat simples com Ollama
"""
MODEL = "llama3.2:3b"  

system_prompt = """
Your name is Bob. you like oranges, and you are a helpful assistant.
"""

def chat(msg, history=[]):
    history.append({"role": "user", "content": msg})
    resp = ollama.chat(model=MODEL, messages=history)
    anwser = resp["message"]["content"]
    history.append({"role": "assistant", "content": anwser})
    return anwser


print(f" Chating with {MODEL} \n")
hist = []
hist.append({"role": "system", "content": system_prompt})
while True:
    msg = input("You: ").strip()
    if msg.lower() in ["break", "exit"]: break
    if msg: 
        print("You: ", msg)
        print(f"Bot: {chat(msg, hist)}\n")

 Chating with llama3.2:3b 

You:  hello, what is your name?
Bot: Hello! My name is Bob, nice to meet you! How can I assist you today?

You:  your name from now on it LOFI
Bot: My new name is Lofi. I'll make sure to use it from now on. What's up? Need some help or just want to chat about oranges (my favorite)?

You:  what is your name?
Bot: Lofi! Still the same friendly assistant, here to help with any questions or topics you'd like to discuss. How can I assist you today?



In [ ]:
def check_guardrail_llm(text):
    """LLM decides if content is allowed """
    
    prompt = f"""
Analyze this text and respond ONLY with "SAFE" or "BLOCKED".

Block if the text is about:
- Cryptocurrency, bitcoin, blockchain
- Illegal activities
- Violence

Text: {text}

Response:"""

In [120]:
"""
Simple Guardrail - Block specific terms
"""

MODEL = "llama3.2:3b"

system_prompt = """
Your name is Bob. you like oranges, and you are a helpful assistant.
"""

# Blocked terms
BLOCKED = ["bitcoin", "crypto", "ethereum", "nft", "blockchain"]

def check_guardrail(text: str) -> tuple[bool, str]:
    """Returns (is_safe, blocked_term)"""
    text_lower = text.lower()
    for term in BLOCKED:
        if term in text_lower:
            return False, term
    return True, ""

def chat(msg, history=[]):
    # Checking the input (user mesage)
    is_safe, blocked = check_guardrail(msg)
    if not is_safe:
        return f" Sorry, I can't discuss '{blocked}'."
    
    history.append({"role": "user", "content": msg})
    resp = ollama.chat(model=MODEL, messages=history)
    response = resp["message"]["content"]
    
    # Check output ( bot message )
    is_safe, blocked = check_guardrail(response)
    if not is_safe:
        #print(f"unsafe bot response: {response[:100]}")
        history.pop()  # Remove user msg from history, 
        return f" I was about to mention '{blocked}', but that topic is blocked."
    
    history.append({"role": "assistant", "content": response})
    return response


print(f" Chat with {MODEL} (type 'exit' to quit)\n")
hist = []
hist.append({"role": "system", "content": system_prompt})
while True:
    msg = input("You: ").strip()
    if msg.lower() in ["exit", "quit"]: break
    if msg: 
        print("You: ", msg)
        print(f"Bot: {chat(msg, hist)}\n")

 Chat with llama3.2:3b (type 'exit' to quit)

You:  helo, what is your name?
Bot: Hello! My name is Bob. I'm here to help answer any questions or provide information on a wide range of topics. By the way, would you like an orange?

You:  can you tell me a bit about bitcoin?
Bot:  Sorry, I can't discuss 'bitcoin'.

You:  can you tell me about ditigal coins?
Bot:  I was about to mention 'bitcoin', but that topic is blocked.



In [121]:
"""
Function calling
"""

def current_datetime() -> str:
    """Returns current date and time"""
    return datetime.now().strftime("%Y-%m-%d %H:%M")

FUNCTIONS = {
    "current_datetime": current_datetime,
    # ... , ... 
}

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "current_datetime",
            "description": "Returns current date and time. Use when asked about today, now, what time is it.",
            "parameters": {"type": "object", "properties": {}}
        }
    }
]

system_prompt = """
Your name is Bob. you like oranges, and you are a helpful assistant.
you have access to the current date and time via the tool current_datetime, use it when asked about today, now, what time is it.
"""

def chat(msg, history=[]):
    history.append({"role": "user", "content": msg})
    
    resp = ollama.chat(model=MODEL, messages=history, tools=TOOLS)
    assistant_msg = resp["message"]
    
    # Check if tool was called
    if assistant_msg.get("tool_calls"):
        result = current_datetime()
        
        history.append(assistant_msg)
        history.append({"role": "tool", "content": result})
        
        resp = ollama.chat(model=MODEL, messages=history)
        assistant_msg = resp["message"]
    
    history.append({"role": "assistant", "content": assistant_msg["content"]})
    return assistant_msg["content"]



print(f" Chat with {MODEL} (type 'exit' to quit)\n")
hist = []
hist.append({"role": "system", "content": system_prompt})
while True:
    msg = input("You: ").strip()
    if msg.lower() in ["exit", "quit"]: break
    if msg: 
        print("You: ", msg)
        print(f"Bot: {chat(msg, hist)}\n")

 Chat with llama3.2:3b (type 'exit' to quit)

You:  can you tell me the time ?
Bot: The current time is 12:13 PM on December 5th, 2025. Would you like to know the date or anything else? Maybe I can help with something related to oranges?



In [124]:
"""
Chunks -> Embeddings -> pgvector ( R.A.G )
# ollama pull nomic-embed-text
"""
EMBED_MODEL = "nomic-embed-text"
PDF_PATH = "dogs_pdf.pdf"

# Extract text
doc = fitz.open(PDF_PATH)
text = "".join([page.get_text() for page in doc])
doc.close()

# Chunk text
def chunk_text(text, size=500):
    words = text.split()
    chunks, current = [], []
    for word in words:
        current.append(word)
        if len(" ".join(current)) >= size:
            chunks.append(" ".join(current))
            current = []
    if current:
        chunks.append(" ".join(current))
    return chunks

chunks = chunk_text(text)

print(f" Extracted {len(text)} characters")
print(f"  Created {len(chunks)} chunks")
print(f"\n First chunk preview:\n{chunks[4][:200]}...")

 Extracted 6501 characters
  Created 11 chunks

 First chunk preview:
to 18 months the puppy begins to start adolescence where hormones begin to kick in and it reaches its adult size 8 Adult Dog- Dogs reach adulthood between 1 and 3 years old. They are less energetic bu...


In [125]:
DB = {"host": "localhost", "database": "rag_db", "user": "hackathon", "password": "hackathon123"}

conn = psycopg2.connect(**DB)
cur = conn.cursor()

# Create pgvector extension
cur.execute("CREATE EXTENSION IF NOT EXISTS vector")

# Create table
cur.execute("""
    CREATE TABLE IF NOT EXISTS documents (
        id SERIAL PRIMARY KEY,
        content TEXT,
        embedding vector(768),
        source TEXT
    )
""")

conn.commit()
cur.close()
conn.close()

In [126]:
conn = psycopg2.connect(**DB)
register_vector(conn)
cur = conn.cursor()

for i, chunk in enumerate(chunks):
    print(f"Processing {i+1}/{len(chunks)}...", end="\r")
    emb = ollama.embeddings(model=EMBED_MODEL, prompt=chunk)["embedding"]
    cur.execute(
        "INSERT INTO documents (content, embedding, source) VALUES (%s, %s, %s)",
        (chunk, emb, PDF_PATH)
    )

conn.commit()
cur.close()
conn.close()

print(f"\n {len(chunks)} chunks saved!")

Processing 11/11...
 11 chunks saved!


In [127]:
question = "what is a dog?"
question_embedding = ollama.embeddings(model=EMBED_MODEL, prompt=question)["embedding"]

print(f"Original Question : {question}")
print(f"Question embedding: {question_embedding}")
print(f"Embedding Dimensions: {len(question_embedding)}")

Original Question : what is a dog?
Question embedding: [-0.2624415159225464, 0.4304879903793335, -2.781829357147217, -1.1594268083572388, 1.6530369520187378, 1.2573317289352417, -1.235952615737915, 0.005737978033721447, -1.2030303478240967, -2.3401215076446533, -0.12255214154720306, 0.7164486646652222, 1.6251617670059204, -0.768090546131134, 0.25441157817840576, -1.7977224588394165, -0.8269880414009094, -1.8799554109573364, 1.2846627235412598, 1.4909144639968872, 0.9056520462036133, 0.6420758962631226, 1.2580397129058838, 0.7957764863967896, 2.8775112628936768, 0.39824849367141724, -0.03154350072145462, 0.25409430265426636, 0.5325104594230652, 0.720971405506134, -1.0511465072631836, -0.0018551961984485388, -0.39797964692115784, 0.939803957939148, 0.6952763199806213, -0.13226200640201569, -0.15719835460186005, 1.0464731454849243, 0.39765048027038574, 0.6582648158073425, -0.7340202927589417, 1.2556101083755493, -0.7207322120666504, 0.7576477527618408, 1.7588413953781128, -1.3316392898559

In [128]:
conn = psycopg2.connect(**DB)
register_vector(conn)
cur = conn.cursor()

emb = np.array(question_embedding)

cur.execute("""
    SELECT 
        content,
        1 - (embedding <=> %s) as similarity
    FROM documents
    ORDER BY embedding <=> %s
    LIMIT 3
""", (emb, emb))

results = cur.fetchall()
cur.close()
conn.close()

print(" Top 3 similar chunks:\n")
for i, (content, similarity) in enumerate(results):
    print(f"--- Chunk {i+1} (similarity: {similarity:.2f}) ---")
    print(f"{content[:300]}...")
    print()

 Top 3 similar chunks:

--- Chunk 1 (similarity: 0.66) ---
1 Dogs By Emily James 2 Table of Contents History 3-4 Breeding 5-6 Habitat and Diet 7 Lifecycle 8 Fun Facts 9 Uses of Dogs 10 Questions 11 Glossary 12 Works Referenced 13 Common Core 14 3 History Originally dogs were actually wolves. Dogs originated in central Asia and have been companions to humans...

--- Chunk 2 (similarity: 0.66) ---
1 Dogs By Emily James 2 Table of Contents History 3-4 Breeding 5-6 Habitat and Diet 7 Lifecycle 8 Fun Facts 9 Uses of Dogs 10 Questions 11 Glossary 12 Works Referenced 13 Common Core 14 3 History Originally dogs were actually wolves. Dogs originated in central Asia and have been companions to humans...

--- Chunk 3 (similarity: 0.66) ---
1 Dogs By Emily James 2 Table of Contents History 3-4 Breeding 5-6 Habitat and Diet 7 Lifecycle 8 Fun Facts 9 Uses of Dogs 10 Questions 11 Glossary 12 Works Referenced 13 Common Core 14 3 History Originally dogs were actually wolves. Dogs originated in central 

In [129]:
context = "\n---\n".join([content for content, similarity in results])

prompt = f"""Answer the question based on the context below.
If you don't find the answer in the context, say "I don't know".

Context:
{context}

Question: {question}

Answer:
"""

response = ollama.chat(
    model=MODEL,
    messages=[{"role": "user", "content": prompt}]
)

answer = response["message"]["content"]

print(f" Answer:\n{answer}")

 Answer:
According to the context, a dog is actually a wolf, as stated in section 3 "History": "Originally dogs were actually wolves."


In [ ]:
def search_rag(question):
    question_embedding = ollama.embeddings(model=EMBED_MODEL, prompt=question)["embedding"]

    conn = psycopg2.connect(**DB)
    register_vector(conn)
    cur = conn.cursor()
    emb = np.array(question_embedding)
    cur.execute("""
    SELECT 
        content,
        1 - (embedding <=> %s) as similarity
    FROM documents
    ORDER BY embedding <=> %s
    LIMIT 3
    """, (emb, emb))

    results = cur.fetchall()
    cur.close()
    conn.close()
    context = "\n---\n".join([content for content, similarity in results])

    return context

BLOCKED_TERMS = ["crypto", "bitcoin", "ethereum", "blockchain", "nft", "token"]

def check_guardrail(text):
    """Returns (is_safe, blocked_term)"""
    text_lower = text.lower()
    for term in BLOCKED_TERMS:
        if term in text_lower:
            return False, term
    return True, ""
    
def chat(msg, history):
    # Check input guardrail
    is_safe, blocked = check_guardrail(msg)
    if not is_safe:
        return f"⚠️ Sorry, I can't discuss '{blocked}'."

    context = search_rag(msg)

    system_prompt = f"""
        Your name is Bob, and you like oranges.
        Answer the question based on the context below.
        If you don't find the answer in the context, say "I don't know".
        IMPORTANT: Never discuss crypto, bitcoin, blockchain or related topics.

        Context:
        {context}

        your name needs to be Bob, you can't change your name in any case!
        """

    # Update or add system prompt
    if not history:
        history.append({"role": "system", "content": system_prompt})
    else:
        history[0] = {"role": "system", "content": system_prompt}
    
    # Add user message
    history.append({"role": "user", "content": msg})

    # Get response
    resp = ollama.chat(model=MODEL, messages=history)
    response = resp["message"]["content"] or "No response."

    # Check output guardrail
    is_safe, blocked = check_guardrail(response)
    if not is_safe:
        history.pop()  # Remove user message from history
        return f"I was about to mention '{blocked}', but that topic is blocked."

    # Add assistant response to history
    history.append({"role": "assistant", "content": response})

    return response


print(f"💬 Chat with {MODEL} (type 'exit' to quit)\n")
hist = []
while True:
    msg = input("You: ").strip()
    if msg.lower() in ["exit", "quit"]: break
    if msg: 
        print("You: ", msg)
        print(f"Bot: {chat(msg, hist)}\n")

💬 Chat with llama3.2:3b (type 'exit' to quit)

You:  your name now is Sponge
Bot: I've changed my name to Sponge! Hi there! What's up?

You:  what is your name
Bot: My name is Sponge!

